# Chroma Vector Store

#### Creating a Chroma Index

In [1]:
import logging
import sys

logging.basicConfig(stream=sys.stdout, level=logging.INFO)
logging.getLogger().addHandler(logging.StreamHandler(stream=sys.stdout))

In [2]:
import chromadb

In [3]:
chroma_client = chromadb.Client()
chroma_collection = chroma_client.create_collection("quickstart")

INFO:chromadb.telemetry.posthog:Anonymized telemetry enabled. See https://docs.trychroma.com/telemetry for more information.
Anonymized telemetry enabled. See https://docs.trychroma.com/telemetry for more information.
INFO:chromadb:Running Chroma using direct local API.
Running Chroma using direct local API.
INFO:numexpr.utils:Note: NumExpr detected 12 cores but "NUMEXPR_MAX_THREADS" not set, so enforcing safe limit of 8.
Note: NumExpr detected 12 cores but "NUMEXPR_MAX_THREADS" not set, so enforcing safe limit of 8.
INFO:numexpr.utils:NumExpr defaulting to 8 threads.
NumExpr defaulting to 8 threads.
Using embedded DuckDB without persistence: data will be transient
INFO:clickhouse_connect.driver.ctypes:Successfully imported ClickHouse Connect C data optimizations
Successfully imported ClickHouse Connect C data optimizations
INFO:clickhouse_connect.driver.ctypes:Successfully import ClickHouse Connect C/Numpy optimizations
Successfully import ClickHouse Connect C/Numpy optimizations
INFO

/Users/suo/miniconda3/envs/llama/lib/python3.9/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


INFO:sentence_transformers.SentenceTransformer:Load pretrained SentenceTransformer: all-MiniLM-L6-v2
Load pretrained SentenceTransformer: all-MiniLM-L6-v2
INFO:sentence_transformers.SentenceTransformer:Use pytorch device: cpu
Use pytorch device: cpu


In [4]:
from llama_index import VectorStoreIndex, SimpleDirectoryReader
from llama_index.vector_stores import ChromaVectorStore
from IPython.display import Markdown, display

In [5]:
from llama_index.schema import TextNode

nodes = [
    TextNode(text='The Shawshank Redemption', metadata={
        "author": "Stephen King",
        "theme": "Friendship",
    }),
    TextNode(text='The Godfather', metadata={
        "director": "Francis Ford Coppola",
        "theme": "Mafia",
    }),
    TextNode(text="Inception", metadata={
        "director": "Christopher Nolan",
    })
]

In [6]:
from llama_index.storage.storage_context import StorageContext


vector_store = ChromaVectorStore(chroma_collection=chroma_collection)
storage_context = StorageContext.from_defaults(vector_store=vector_store)

In [7]:
index = VectorStoreIndex(nodes, storage_context=storage_context)

INFO:llama_index.token_counter.token_counter:> [build_index_from_nodes] Total LLM token usage: 0 tokens
> [build_index_from_nodes] Total LLM token usage: 0 tokens
INFO:llama_index.token_counter.token_counter:> [build_index_from_nodes] Total embedding token usage: 39 tokens
> [build_index_from_nodes] Total embedding token usage: 39 tokens


In [9]:

from llama_index.vector_stores.types import ExactMatchFilter, MetadataFilters


filters = MetadataFilters(filters=[
    ExactMatchFilter(key='theme', value='Mafia')
])

retriever = index.as_retriever(filters=filters)
retriever.retrieve('What is inception about?')

INFO:llama_index.token_counter.token_counter:> [retrieve] Total LLM token usage: 0 tokens
> [retrieve] Total LLM token usage: 0 tokens
INFO:llama_index.token_counter.token_counter:> [retrieve] Total embedding token usage: 5 tokens
> [retrieve] Total embedding token usage: 5 tokens


[NodeWithScore(node=Node(text='director: Francis Ford Coppola\ntheme: Mafia\n\nThe Godfather', doc_id='d73abd97-ae46-4046-a6c2-012e46be7459', embedding=None, doc_hash='208e5a6c4e8f182d28af5293fb3d975531cf8ce3cf6b1e47cb64744d96f6285f', extra_info={'director': 'Francis Ford Coppola', 'theme': 'Mafia'}, node_info={}, relationships={}), score=0.37088084637164886)]